# Training linear models
In this notebook, we will first start by looking at the Linear Regression model. The model will be trained in 2 ways:
1. Using a direct 'closed-form' equation - that directly computes the model parameters best fit to the training set.
2. Using an iterative optimization approach called Gradient Descent(GD) - that gradually tweaks the model parameters to minimize the cost function over the training set, eventually converging on the same set of parameters as the first method.

Then we will look at Polynomial Regression, a model that can fit non-linear datasets.
<br><br>
Then we will look at several Regularization techniques that can reduce the risk of overfitting the training sets.
<br><br>
Finally, we will look at 2 more models : Logistic Regression and Softmax Regression.

## Linear Regression

### Using the normal equation